# Playingcard Generator

## Imports

In [ ]:
import os
import logging

import prettymaps

import pandas as pd
from svglib.svglib import svg2rlg

from pypdf import PdfReader, PdfWriter
from pdfrw import PdfReader as PageReader, PageMerge
from pdfrw.buildxobj import pagexobj
from pdfrw.toreportlab import makerl

from reportlab.pdfgen import canvas
from reportlab.graphics import renderPDF
from reportlab.lib.colors import HexColor

## Configuration

In [ ]:
# Dimension of playing card
CARD_WIDTH, CARD_HEIGHT = 500, 683

# Radius parameter for map generation
MAP_RADIUS = 400

# Output directory
OUTPUT_DIR = "cards_berlin"

# Temporary directory and filenames
TMP_DIR = "tmp"
MAP_FILE = f"{TMP_DIR}/map.pdf"
CROP_FILE = f"{TMP_DIR}/cropped.pdf"

# Assets
ASSETS_DIR = "assets"
LOCATIONS_FILE = f"{ASSETS_DIR}/locations_berlin.csv"
MASK_FILE = f"{ASSETS_DIR}/mask.svg"

# Text color & size
PRIMARY_TEXT_COLOR = 0x2F3737
PRIMARY_FONT_TYPE = "Times-Bold"
SECONDARY_TEXT_COLOR = 0x646464
SECONDARY_FONT_TYPE = "Times-Italic"

In [ ]:
# Suppress warnings from matplotlib
logging.getLogger('matplotlib').setLevel(logging.ERROR)

## Helper functions

In [ ]:
def get_primary_font_size(text):
    font_sizes = {
        (0, 15): 50,
        (15, 19): 45,
        (19, 23): 38,
        (23, 28): 34,
        (28, 32): 30,
        (32, 36): 28,
    }
    for (start, end), size in font_sizes.items():
        if start <= len(text) < end:
            return size
    return 25

def get_secondary_font_size(text):
    font_sizes = {
        (0, 30): 30,
        (30, 40): 22,
        (40, 45): 20,
    }
    for (start, end), size in font_sizes.items():
        if start <= len(text) < end:
            return size
    return 17

In [ ]:
def crop_to_card_size():
    reader = PdfReader(MAP_FILE)
    pdf_writer = PdfWriter()

    # Get the first page
    page = reader.pages[0]

    crop_x = (page.mediabox.right - CARD_WIDTH) / 2
    crop_y = (page.mediabox.top - CARD_HEIGHT) / 2

    lower_left_x = crop_x
    lower_left_y = crop_y
    upper_right_x = page.mediabox.right - crop_x
    upper_right_y = page.mediabox.top - crop_y

    cropbox = (lower_left_x, lower_left_y, upper_right_x, upper_right_y)

    page.cropbox.lower_left = cropbox[0:2]
    page.cropbox.upper_right = cropbox[2:4]

    # Add the cropped page to the new PDF
    pdf_writer.add_page(page)

    # Save the cropped PDF to a file
    with open(CROP_FILE, 'wb') as output_file:
        pdf_writer.write(output_file)


In [ ]:
def generate_playing_card(location, title, subtitle=None):
    """
    Generate a playing card with a map, title, and subtitle.

    Args:
        location (str): Location for the map.
        title (str): Title of the playing card.
        subtitle (str): Subtitle of the playing card.

    Returns:
        None
    """
    
    # Generate map plot
    plot = prettymaps.plot(location, radius=MAP_RADIUS, show=False)

    # Create tmp directory, if it doesn't exist
    if not os.path.exists(TMP_DIR):
        os.makedirs(TMP_DIR)

    # Create output directory, if it doesn't exist
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)

    # Save map plot & crop to correct size
    plot.fig.savefig(MAP_FILE, transparent=False)
    crop_to_card_size()

    # Set filename and primary text offset
    if subtitle:
        filename = f"{title}-{subtitle}"
        offset = 122
    else:
        filename = title
        offset = 140

    filename = "".join([c for c in filename if c.isalpha() or c.isdigit() or c == '-']).rstrip()

    # Open canvas image
    c = canvas.Canvas(f"{OUTPUT_DIR}/{filename}.pdf", pagesize=(CARD_WIDTH, CARD_HEIGHT))
    
    # Add map background
    input_pdf = PageReader(CROP_FILE)
    page = input_pdf.pages[0]
    page_obj = PageMerge().add(page)[0]
    c.doForm(makerl(c, page_obj))
    
    # Add overlay
    renderPDF.draw(svg2rlg(MASK_FILE), c, 0, 0)

    # Add primary text
    c.setFont(PRIMARY_FONT_TYPE, get_primary_font_size(title))
    c.setFillColor(HexColor(PRIMARY_TEXT_COLOR))
    c.drawCentredString(CARD_WIDTH / 2, CARD_HEIGHT - offset, title)

    # Add secondary text
    if subtitle:
        c.setFont(SECONDARY_FONT_TYPE, get_secondary_font_size(subtitle))
        c.setFillColor(HexColor(SECONDARY_TEXT_COLOR))
        c.drawCentredString(CARD_WIDTH / 2, CARD_HEIGHT - 170, subtitle)
    
    # Close canvas image
    c.save()


## Generate all playingcards

In [ ]:
def generate_all_playing_cards():
    """
    Generates playing cards for each location in the CSV file.
    The CSV file should contain columns for 'Generated', 'OSM', 'Name', and 'Beschreibung'.
    For each location, if 'Generated' is not True and 'OSM' is not null, a card is generated.
    After generating the card, the 'Generated' column is updated to True in the CSV file.
    """
    # Read csv file in form of list of lists
    locations = pd.read_csv(LOCATIONS_FILE)

    for index, row in locations.iterrows():
        generated = row['Generated']
        location = row['OSM']
        name = row['Name']
        description = row['Beschreibung']

        if pd.notnull(generated) and not bool(generated) and pd.notnull(location):
            print("Generating card for", name, "...")

            if pd.notnull(description):
                generate_playing_card(location, name, description)
            else:
                generate_playing_card(location, name, None)

            locations.at[index, 'Generated'] = True
            locations.to_csv(LOCATIONS_FILE, index=False)

generate_all_playing_cards()


## Reset generation progress

In [ ]:
def reset_generated_locations():
    # Read csv file
    locations = pd.read_csv(LOCATIONS_FILE)

    # Update 'Generated' column to False for all rows
    locations['Generated'] = False

    # Save updated DataFrame to csv file
    locations.to_csv(LOCATIONS_FILE, index=False)

# After all cards have been generated, the generation progress will be reset
reset_generated_locations()